<a href="https://colab.research.google.com/github/Yamito548/Trabajos-cursos-IA-y-big-data/blob/main/TAREA_8_RNN_USANDO_EMBEDDINGS_(TRAFALGAR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerías

In [1]:
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import random
import io

Importamos las librerías que vamos a utilizar en este trabajo. Algunas de las más comunes que solemos hacer.

# Descargamos el Quijote

Con este código, descargamos un archivo txt sobre El Quijote.

In [2]:
path = keras.utils.get_file(
  fname="/content/don_quijote2.txt",
  origin="https://raw.githubusercontent.com/Yamito548/Trabajos-cursos-IA-y-big-data/main/quijote.txt"
)
path

'/content/don_quijote2.txt'

# Manipulamos los datos

Para que el modelo entrene mucho mejor, vamos a hacer que todas las letras sean minúsculas. Así el modelo no tendrá que aprender a diferenciar entre mayúsculas y minúsculas.

In [3]:
text = open(path).read().lower()
#Comprobamos la longitud del texto.
print("Longitud del texto: {}".format(len(text)))
#También vemos un ejemplo del texto.
print(text[0:300])

Longitud del texto: 2097953
el ingenioso hidalgo don quijote de la mancha


tasa

yo, juan gallo de andrada, escribano de cámara del rey nuestro señor, de
los que residen en su consejo, certifico y doy fe que, habiendo visto por
los señores dél un libro intitulado el ingenioso hidalgo de la mancha,
compuesto por miguel de cerv


Podemos ver que el texto efectivamente, solo presenta minúsculas.

En esta parte del código, lo que hacemos es revisar el Quijote en busca de carácteres, los cuales no puedes dar problemas a la hora de trabajar con el modelo, además de que no nos sirven para nada. Lo que hace este código es recorrer el texto del Quijote, si la letra que lea no es un carácter, lo meterá en la nueva variable llamada texto. Ya luego, hacemos que text tenga el contenido de texto, es decir sin carácteres.

In [4]:
texto=""
for letra in text:
  if not letra in "?¿.,;'¡!()-{};[]»«:\"":
    texto+=letra
text=texto

Seguimos con la manipulación, y en este caso lo que haremos será dividir el texto en palabras. Además, se crea otra variable en donde se almacenan todas las palabras únicas. Y luego se crea un diccionario con esta última variable.

In [5]:
palabras = text.split()
palabras_unicas = set(palabras)
word_index = {}
k=0
for palabra in palabras_unicas:
  word_index[palabra] = k
  k+=1

# Descargamos el nltk

Esto es una librería para el procesamiento del lenguaje natural (NLP). Por otro lado, punkt es un modelo de tokenización.

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Además de este diccionario, creamos su inverso.

In [7]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
#Revisamos el tamaño de la variable de palabras
len(palabras)

381215

# Obtención de secuencias de entrada y palabra a predecir

Ahora vamos a obtener las sequencias de entrada y la palabra a predecir.

In [8]:
#Definimos el tamaño de las secuencias.
SEQ_LENGTH = 5
step = 1 #Para que la siguiente palabra este a la izquierda

sequences = []
next_words = []

En este código, recorremos la variable de las palabras. Con un rango entre 0 y la cantidad de palabras que hay. Restándole el SEQ_LENGTH y indicando de que debe seguir a la izquierda.

In [9]:
for i in range(0,len(palabras)-SEQ_LENGTH, step):
  sequences.append(palabras[i:i+SEQ_LENGTH])
  next_words.append(palabras[i+SEQ_LENGTH])


#Vemos las secuencias, ejemplo
sequences[7]
#Y la siguiente palabra
next_words[7]

'de'

Vamos a reducir el modelo, debido a que el quijote es muy grande y podría llevarnos a problemas de tiempo. Pero ten en cuenta que a menos datos, peor será el modelo. Esto no quiere decir que si quitas 1000 palabras vaya a ser muy malo, me refiero a que tampoco te pases quitando datos. Por ejemplo, usar el 70-90% porciento creo que estaría bien, y usar menos de eso podría llevar a que el modelo no sea tan preciso.

In [10]:
#Cantidad máxima de secuencias
MAX_SEQUENCES = 31000

#Se reorganiza de forma aleatoria los indices.
perm = np.random.permutation(len(sequences))
#Convertimos a una array numpy
sequences, next_words = np.array(sequences), np.array(next_words)
sequences, next_words = sequences[perm], next_words[perm]
#Reducción del tamaño
sequences, next_words = list(sequences[:MAX_SEQUENCES]), list(next_words[:MAX_SEQUENCES])

En este código, por cada secuencia se crea una cadena de palabras, luego se agrega a la lista de frases. Ya por último, divide la frase en palabras individuales.

In [11]:
#Convertimos las sequencias en frases
frases = []
for i in range(len(sequences)):
  frase = ' '.join(sequences[i])
  frases.append(frase)
#Tokenizar las frases
newsVec = [nltk.word_tokenize(frase) for frase in frases]
newsVec

[['hasta', 'los', 'encantadores', 'son', 'médicos'],
 ['que', 'debes', 'a', 'nuestra', 'amistad'],
 ['tal', 'retablo', 'tenía', 'una', 'varilla'],
 ['hasta', 'los', 'encantados', 'no', 'perdona'],
 ['cual', 'gustando', 'de', 'oírle', 'decir'],
 ['que', 'tomando', 'don', 'fernando', 'una'],
 ['armas', 'señor', 'respondió', 'sancho', 'no'],
 ['merced', 'quiere', 'un', 'traguito', 'aunque'],
 ['las', 'entrañas', 'y', 'por', 'más'],
 ['costillas', 'quebradas', 'y', 'aunque', 'tonto'],
 ['luscinda', 'un', 'libro', 'de', 'caballerías'],
 ['aldea', 'y', 'aun', 'a', 'todos'],
 ['hicieron', 'titubear', 'en', 'su', 'propósito'],
 ['señor', 'que', 'ayer', 'o', 'antes'],
 ['añadidura', 'estas', 'dos', 'pistolas', 'y'],
 ['se', 'afrontara', 'a', 'fee', 'que'],
 ['baja', 'le', 'dijo', 'todas', 'estas'],
 ['dellos', 'la', 'mitad', 'de', 'los'],
 ['me', 'place', 'respondió', 'el', 'mozo'],
 ['haya', 'alguno', 'dellos', 'tomado', 'su'],
 ['lo', 'menos', 'a', 'mí', 'tales'],
 ['se', 'quietó', 'y', 'pros

En este caso, me di cuenta de que en uno de las características que no se habían filatrado era el punto y coma, me di cuenta y antes de seguir pues lo añadí a la lista de características no deseadas.

# Creamos el modelo

Importamos el modelo word2vec

In [12]:
from gensim.models import Word2Vec
model = Word2Vec(newsVec,min_count=1,vector_size=32)

Aquí también me dio un fallo, aunque era más bien por temas de versión. En este caso, tuve que cambiar el parámetro **size** por **vector size**. Por otro lado, no entendía para que servía el **min_count** y bueno me parece importante indicar que es un parámetro que indica cuantas veces debe aparecer una palabra para poder ser incluida en el modelo, en este caso por defecto esta puesta en 1, es decir que todas pueden ser usadas.

In [13]:
#Vemos el vector de cualquier palabra
model.wv["caballero"]

array([-0.43750364, -0.8821288 ,  0.636035  ,  0.7817121 ,  0.37992623,
       -0.6697675 ,  0.9843641 ,  0.4509089 , -0.26649204,  0.3525724 ,
        0.75283694, -0.7382355 ,  0.5252056 , -0.6217387 , -0.24697876,
        0.39434132, -0.22025648,  0.8736286 , -0.30933324,  0.3845086 ,
        1.1636933 ,  1.5162797 ,  1.7234327 , -0.085145  ,  0.4494232 ,
        0.04075314, -1.1647332 ,  0.06514525,  0.45351535, -1.0334464 ,
       -0.4744515 ,  0.5249122 ], dtype=float32)

Ahora toca una de las partes que más me gustan, y es la de ver las palabras similares

In [14]:
model.wv.most_similar("caballero")

[('gobernador', 0.9958556890487671),
 ('mío', 0.9957267642021179),
 ('duque', 0.9950684309005737),
 ('sansón', 0.9943887591362),
 ('dios', 0.992843747138977),
 ('famoso', 0.9925079345703125),
 ('diciéndole', 0.9917805790901184),
 ('preguntó', 0.9917787909507751),
 ('hidalgo', 0.9917699694633484),
 ('pues', 0.9915176630020142)]

Como vemos, muchas de las palabras que vemos concuerdan con la palabra caballero, como lo es caballería o hombre. En esta línea tuve otro problema, esta vez con la versión del gensim, lo arreglé y funcionó enseguida.

# Definición de X e y

Para esta tarea, usaremos embedings.

In [15]:
NUM_SEQUENCES = len(sequences)
X = np.zeros((NUM_SEQUENCES, SEQ_LENGTH, 32))
y = np.zeros((NUM_SEQUENCES, len(palabras_unicas)))

In [16]:
#Codificación de la y
for k in range(NUM_SEQUENCES):
  palabra=next_words[k]
  y[k,word_index[palabra]]=1 #codificación ONE-HOT

In [17]:
#Codificación de la X
for k in range(NUM_SEQUENCES):
  for w in range(SEQ_LENGTH):
    word=sequences[k][w]
    X[k,w]=model.wv[word]

# Terminamos el modelo

Ya queda poco para terminar, lo siguiente a hacer es definir el modelo que vamos a utilizar como lo son su tipo, tamaño y capas. En este caso, solo hay 2 capas.

In [18]:
vocabulary_size=len(palabras_unicas)
model=Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH,32)))
model.add(Dense(vocabulary_size,activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               82432     
                                                                 
 dense (Dense)               (None, 22949)             2960421   
                                                                 
Total params: 3042853 (11.61 MB)
Trainable params: 3042853 (11.61 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


Compilamos el modelo antes de entrenarlo. Usaremos el optimizador adam para esta tarea y la metrica para saber si el modelo va bien será el accuracy.

In [19]:
optimizer='adam'
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])

Entrenamos el modelo, esto puede tardar un poco. Usamos como variables de los datos la X y y que creamos antes, con un poquito de porcentaje para los datos de validación, con solo 1 epoca y por último que los datos esten mesclados para un mejor entrenamiento.

In [20]:
history=model.fit(X,y, validation_split=0.05, batch_size=128, epochs=1,shuffle=True)

231/231 [==============================] - 36s 144ms/step - loss: 7.2415 - accuracy: 0.0506 - val_loss: 6.8800 - val_accuracy: 0.0574
